# AutoGen Grundlæggende Eksempel

I dette kodeeksempel vil du bruge [AutoGen](https://aka.ms/ai-agents/autogen) AI-rammeværket til at oprette en grundlæggende agent.

Målet med dette eksempel er at vise dig de trin, som vi senere vil bruge i de yderligere kodeeksempler, når vi implementerer de forskellige agentmønstre.


## Importer de nødvendige Python-pakker


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Opret klienten

I dette eksempel vil vi bruge [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for at få adgang til LLM.

`model` er defineret som `gpt-4o-mini`. Prøv at ændre modellen til en anden model, der er tilgængelig på GitHub Models-markedspladsen, for at se forskellige resultater.

Som en hurtig test vil vi blot køre en simpel prompt - `Hvad er hovedstaden i Frankrig`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definere agenten

Nu hvor vi har opsat `client` og bekræftet, at den fungerer, lad os oprette en `AssistantAgent`. Hver agent kan tildeles:  
**name** - Et kort navn, der er nyttigt til at referere til den i multi-agent flows.  
**model_client** - Den klient, du oprettede i det tidligere trin.  
**tools** - Tilgængelige værktøjer, som agenten kan bruge til at udføre en opgave.  
**system_message** - Metaprompten, der definerer opgaven, adfærden og tonen for LLM.  

Du kan ændre systemmeddelelsen for at se, hvordan LLM reagerer. Vi vil dække `tools` i Lektion #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Kør agenten

Funktionen nedenfor vil køre agenten. Vi bruger metoden `on_message` til at opdatere agentens tilstand med den nye besked.

I dette tilfælde opdaterer vi tilstanden med en ny besked fra brugeren, som er `"Planlæg en fantastisk solrig ferie"`.

Du kan ændre beskedens indhold for at se, hvordan LLM reagerer forskelligt.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os ikke ansvar for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
